In [1]:
import pandas as pd 
import numpy as np 
import datetime
import sqlite3 
import pybaseball as pyb #Pybaseball is where we will be pulling data from

In [3]:
# Calculate yesterday's date
today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)

In [4]:
#turn the output of the date to a string to be able to pass through the statcast function
yesterday_str = yesterday.strftime('%Y-%m-%d')
#Get yesterdays data from pybaseball's statcast function 
data = pyb.statcast(start_dt=yesterday_str, end_dt=None, team=None, verbose=True, parallel=True)

This is a large query, it may take a moment to complete
Skipping offseason dates


0it [00:00, ?it/s]


In [73]:
def add_to_savant(df):
    # Modify 'pfx_x' and 'pfx_z' columns to get them in inches 
    df['pfx_x'] = df['pfx_x'].apply(lambda x: x * -12)
    df['pfx_z'] = df['pfx_z'].apply(lambda x: x * 12)
    
    # Create 'hard_hit' column
    df['launch_speed'] = df['launch_speed'].fillna(False)
    df['hard_hit'] = df['launch_speed'].apply(lambda x: 1 if x > 95 else 0)

    # Create 'sweet_spot' column
    df['launch_angle'] = df['launch_angle'].fillna(False)
    df['sweet_spot'] = df['launch_angle'].apply(lambda x: 1 if (x >= 8) and (x <= 32) else 0)

    # Add 'VAA' column
    vy_f = -1 * np.sqrt(df['vx0']**2 + df['vy0']**2) 
    t = (vy_f - df['vy0']) / df['ay']
    vz_f = df['vz0'] + df['az'] * t
    df['VAA'] = np.round(-1 * np.arctan(vz_f / vy_f) * (180 / np.pi), 2)
    
    return df

In [40]:
# Define a dictionary with the column name changes
column_names_dict = {
    'release_speed': 'velo',
    'release_pos_x': 'release_side',
    'release_pos_z': 'release_height',
    'game_date': 'date',
    'pf_x_z': 'ind_vert_break',
    'pf_x_x': 'horizontal_break',
    'launch_speed': 'exit_velo',
    
    # Add more column names as needed
}

In [46]:
def rename_statcast_columns(data, column_names_dict):
    """
    Rename columns from the Statcast data that was pulled from Savant

    Parameters:
        data (pd.DataFrame): Statcast data from Savant
        column_names_dict (dict): a dictionary containing the current column names as keys
                                  and the new column names as values

    Returns:
        pd.DataFrame: the modified DataFrame with the renamed columns
    """
    data = data.rename(columns=column_names_dict)
    return data

In [ ]:
# Run the downloaded data from pybaseball for the day through the 'add_to_savant' function
modified_data = add_to_savant(data)

In [ ]:
# Run the new dataframe (modified_data) that contains the new columns from the 'add_to_savant' function through 
# the function that renames the columns
data_cleaned = rename_columns(modified_data, column_names_dict)

In [ ]:
#Connect to the SQLLite database 2023Statcast
conn = sqlite3.connect('2023Statcast.db')

#define the table name
table_name = 'statcast_data_2023'
#if the table name exists then append the data on it
data_cleaned.to_sql(table_name, conn, if_exists='append', index=False)

# Close the connection
conn.close()


TESTING THE CODE

In [25]:
x = pd.read_csv('/Users/emiliomartinez/Desktop/code/Data/Savant/2022savant.csv').head(10)


In [42]:
z = add_to_savant(x)

In [47]:
q= rename_statcast_columns(z,column_names_dict)

In [81]:
#Connect to the SQLLite database 2023Statcast
conn = sqlite3.connect('2023Statcast.db')

#define the table name
table_name = 'statcast_data_2023'
#if the table name exists then append the data on it
q.to_sql(table_name, conn, if_exists='append', index=False)

# Close the connection
conn.close()


In [78]:
conn = sqlite3.connect('2023Statcast.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM statcast_data_2023;")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

OperationalError: no such table: statcast_data_2023

In [77]:
conn = sqlite3.connect('2023Statcast.db')
cursor = conn.cursor()

cursor.execute("DROP TABLE statcast_data_2023;")
conn.commit()

conn.close()

In [82]:
# Connect to the database
conn = sqlite3.connect('2023Statcast.db')
cursor = conn.cursor()

# Execute the query to retrieve the column names
cursor.execute("PRAGMA table_info(statcast_data_2023);")

# Fetch the results and print the column names
results = cursor.fetchall()
for row in results:
    print(row[1])

# Close the database connection
conn.close()

pitch_type
date
velo
release_side
release_height
player_name
batter
pitcher
events
description
spin_dir
spin_rate_deprecated
break_angle_deprecated
break_length_deprecated
zone
des
game_type
stand
p_throws
home_team
away_team
type
hit_location
bb_type
balls
strikes
game_year
pfx_x
pfx_z
plate_x
plate_z
on_3b
on_2b
on_1b
outs_when_up
inning
inning_topbot
hc_x
hc_y
tfs_deprecated
tfs_zulu_deprecated
fielder_2
umpire
sv_id
vx0
vy0
vz0
ax
ay
az
sz_top
sz_bot
hit_distance_sc
exit_velo
launch_angle
effective_speed
release_spin_rate
release_extension
game_pk
pitcher.1
fielder_2.1
fielder_3
fielder_4
fielder_5
fielder_6
fielder_7
fielder_8
fielder_9
release_pos_y
estimated_ba_using_speedangle
estimated_woba_using_speedangle
woba_value
woba_denom
babip_value
iso_value
launch_speed_angle
at_bat_number
pitch_number
pitch_name
home_score
away_score
bat_score
fld_score
post_away_score
post_home_score
post_bat_score
post_fld_score
if_fielding_alignment
of_fielding_alignment
spin_axis
delta_home_win_